In [1]:
from google.cloud import bigquery
import pandas as pd

In [2]:
client = bigquery.Client(project='product-analytics-portfolio')

#### Prepping for the initial data pipeline

In [ ]:
test_query = """WITH stacked_table AS (SELECT event_date,
       event_timestamp,
       user_pseudo_id,
       event_name
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20210130`
WHERE event_name IN ('page_view', 'add_to_cart', 'begin_checkout', 'purchase')
UNION ALL
SELECT event_date,
       event_timestamp,
       user_pseudo_id,
       event_name
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20210131`
WHERE event_name IN ('page_view', 'add_to_cart', 'begin_checkout', 'purchase')),

flagged_events AS (SELECT *,
       CASE WHEN event_name = 'page_view' THEN 1 ELSE 0 END AS page_view,
       CASE WHEN event_name = 'add_to_cart' THEN 1 ELSE 0 END AS add_to_cart,
       CASE WHEN event_name = 'begin_checkout' THEN 1 ELSE 0 END as begin_checkout,
       CASE WHEN event_name = 'purchase' THEN 1 ELSE 0 END as purchase
FROM stacked_table)

SELECT user_pseudo_id,
       MIN(event_date) AS first_event_date,
       MIN(event_timestamp) AS first_event_timestamp,
       CASE WHEN SUM(page_view) > 0 THEN 1 ELSE 0 END viewed_page,
       CASE WHEN SUM(add_to_cart) > 0 THEN 1 ELSE 0 END added_to_cart,
       CASE WHEN SUM(begin_checkout) > 0 THEN 1 ELSE 0 END began_checkout,
       CASE WHEN SUM(purchase) > 0 THEN 1 ELSE 0 END purchased
FROM flagged_events
GROUP BY user_pseudo_id;"""
test_results = client.query(test_query).to_dataframe()